In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import shutil
from matplotlib import pyplot as plt

In [2]:
src_path = '../img_clip_all'
dst_path = './img_stitch_all'
material_path = './img_material_all'

In [3]:
if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.mkdir(dst_path)
if os.path.exists(material_path):
    shutil.rmtree(material_path)
os.mkdir(material_path)

In [ ]:
# Move test data

In [17]:
df_test = pd.read_csv('../test_day6.csv')

In [18]:
for filename, df_cur in df_test.groupby('filename_gfp'):
    shutil.copyfile(os.path.join(src_path, filename), 
                   os.path.join(dst_path, filename))

In [19]:
df_ori = pd.read_csv('../train_day6.csv')

In [20]:
for filename, df_cur in df_ori.groupby('filename_gfp'):
    shutil.copyfile(os.path.join(src_path, filename), 
                   os.path.join(dst_path, filename))

In [10]:
# df_ori

In [6]:
df_material_obj = []
df_material_img = []
for filename, df_cur in df_ori.groupby('filename_gfp'):
    img = cv2.imread(os.path.join(src_path, filename))
    for i in range(len(df_cur)):
        x1 = df_cur.iloc[i]['x1']
        y1 = df_cur.iloc[i]['y1']
        x2 = df_cur.iloc[i]['x2']
        y2 = df_cur.iloc[i]['y2']
        obj_img = img[y1: y2, x1: x2, :]
        df_material_obj.append(list(df_cur.iloc[i]))
        cv2.imwrite(os.path.join(material_path, filename.replace('_gfp', '_' + str(df_cur.iloc[i]['bbox_idx']))), obj_img)
        img[y1: y2, x1: x2, :] = 0
        
    df_material_img.append(list(df_cur.iloc[i]))
    cv2.imwrite(os.path.join(material_path, filename.replace('_gfp', '')), img)
    
df_material_obj = pd.DataFrame(data=df_material_obj, columns=df_ori.columns)
df_material_img = pd.DataFrame(data=df_material_img, columns=df_ori.columns)

In [7]:
df_material_obj.to_csv('./material_obj.csv', index=False, sep=',')
df_material_img.to_csv('./material_img.csv', index=False, sep=',')

In [8]:
# obj_img = cv2.cvtColor(obj_img, cv2.COLOR_BGR2RGB)
# plt.imshow(obj_img)

In [9]:
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)

In [9]:
df_material_obj = pd.read_csv('./material_obj.csv', sep=',')
df_material_img = pd.read_csv('./material_img.csv', sep=',')

In [10]:
# if os.path.exists(dst_path):
#     shutil.rmtree(dst_path)
# os.mkdir(dst_path)

In [11]:
# df_material_obj

In [12]:
# df_material_img

In [13]:
def check_cover(res_cur_list, pos):
    flag = True
    x1, y1, x2, y2 = pos
    for i in range(len(res_cur_list)):
        _x1, _y1, _x2, _y2 = res_cur_list[i][8:12]
        if (x1>=_x1 and x1<=_x2 and y1>=_y1 and y1<=_y2) or \
        (x1>=_x1 and x1<=_x2 and y2>=_y1 and y2<=_y2) or \
        (x2>=_x1 and x2<=_x2 and y1>=_y1 and y1<=_y2) or \
        (x2>=_x1 and x2<=_x2 and y2>=_y1 and y2<=_y2):
            flag = False
            break
    return flag

In [14]:
df_res = []
# 每个图片重复n次
image_id = 10000
res_all_list = []
for epoch in range(3):
    for i in range(len(df_material_img)):
        img = cv2.imread(os.path.join(material_path, df_material_img.iloc[i]['filename']))
        height, width = img.shape[:2]
        
        res_cur_list = []
        # 随机采样100个样本
        df_sample = df_material_obj.sample(30)
        for j in range(len(df_sample)):
            x1 = np.random.randint(0, width - df_sample.iloc[j]['width'] + 1)
            y1 = np.random.randint(0, height - df_sample.iloc[j]['height'] + 1)
            x2 = x1 + df_sample.iloc[j]['width']
            y2 = y1 + df_sample.iloc[j]['height']
            if check_cover(res_cur_list, [x1, y1, x2, y2]):
                
                obj_img = cv2.imread(os.path.join(material_path, 
                                                  df_sample.iloc[j]['filename'].replace('.', '_' + str(df_sample.iloc[j]['bbox_idx']) + '.')))
                img[y1: y2, x1: x2, :] = obj_img
                
                cur_data = df_sample.iloc[j].tolist()
                cur_data[5] = df_material_img.iloc[i]['filename'].replace('.', '_' + str(epoch) + '.')
                cur_data[6] = cur_data[5].replace('.', '_gfp.')
                cur_data[7] = cur_data[5].replace('.', '_ph.')
                cur_data[8] = x1
                cur_data[9] = y1
                cur_data[10] = x2
                cur_data[11] = y2
                cur_data[12] = cur_data[-3]
                cur_data[13] = image_id
                cur_data = cur_data[:12 + 2]
                res_cur_list.append(cur_data)
        # 存图
        cv2.imwrite(os.path.join(dst_path, cur_data[6]), img)
        image_id += 1
        res_all_list += res_cur_list
df_res = pd.DataFrame(data=res_all_list, columns=list(df_material_img.columns[:12]) + ['batch', 'image_id'])

In [15]:
df_res.to_csv('./train_stitch.csv', index=False, sep=',')

In [21]:
df_res_all = pd.concat((df_res, df_ori[df_res.columns]))
df_res_all = df_res_all.reset_index(drop=True)
df_res_all['bbox_idx'] = df_res_all.index

In [22]:
df_res_all.to_csv('./train_stitch_all.csv', index=False, sep=',')

In [23]:
df_res_all

,day,label,size,width,height,filename,filename_gfp,filename_ph,x1,y1,x2,y2,batch,image_id,bbox_idx
0,6,paraclone10,1344,111,113,1635560473_382136_0.png,1635560473_382136_0_gfp.png,1635560473_382136_0_ph.png,1106,233,1217,346,4,10000,0
1,6,paraclone17,1344,40,48,1635560473_382136_0.png,1635560473_382136_0_gfp.png,1635560473_382136_0_ph.png,399,597,439,645,5,10000,1
2,6,holoclone5,1344,205,223,1635560473_382136_0.png,1635560473_382136_0_gfp.png,1635560473_382136_0_ph.png,816,418,1021,641,5,10000,2
3,6,meroclone2,1344,95,119,1635560473_382136_0.png,1635560473_382136_0_gfp.png,1635560473_382136_0_ph.png,495,931,590,1050,5,10000,3
4,6,paraclone18,1344,52,65,1635560473_382136_0.png,1635560473_382136_0_gfp.png,1635560473_382136_0_ph.png,989,833,1041,898,6,10000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160855,6,paraclone24,1344,108,103,1635563657_019476.png,1635563657_019476_gfp.png,1635563657_019476_ph.png,754,666,862,769,9,2565,160855
160856,6,paraclone24,1344,108,103,1635563658_988402.png,1635563658_988402_gfp.png,1635563658_988402_ph.png,496,520,604,623,9,2566,160856
160857,6,paraclone24,1344,108,103,1635563661_0426219.png,1635563661_0426219_gfp.png,1635563661_0426219_ph.png,488,242,596,345,9,2567,160857
160858,6,paraclone24,1344,108,103,1635563663_13599.png,1635563663_13599_gfp.png,1635563663_13599_ph.png,173,666,281,769,9,2568,160858


In [18]:
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)